In [1]:
# My repo
!git clone https://github.com/Tixonmavrin/covid-19-solution
# Data world
!git clone https://github.com/CSSEGISandData/COVID-19.git
# Data world new
!git clone https://github.com/CSSEGISandData/COVID-19 -b web-data COVID-19-new
# Data Russia
!git clone https://github.com/grwlf/COVID-19_plus_Russia.git
# More data
!wget https://coronadatascraper.com/timeseries.csv

Cloning into 'covid-19-solution'...
remote: Enumerating objects: 67, done.
remote: Counting objects: 100% (67/67), done.
remote: Compressing objects: 100% (66/66), done.
remote: Total 164 (delta 29), reused 0 (delta 0), pack-reused 97
Receiving objects: 100% (164/164), 5.26 MiB | 4.04 MiB/s, done.
Resolving deltas: 100% (59/59), done.
Cloning into 'COVID-19'...
remote: Enumerating objects: 28, done.
remote: Counting objects: 100% (28/28), done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 25054 (delta 15), reused 22 (delta 13), pack-reused 25026
Receiving objects: 100% (25054/25054), 112.70 MiB | 12.32 MiB/s, done.
Resolving deltas: 100% (13930/13930), done.
Cloning into 'COVID-19-new'...
remote: Enumerating objects: 28, done.
remote: Counting objects: 100% (28/28), done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 25054 (delta 15), reused 22 (delta 13), pack-reused 25026
Receiving objects: 100% (25054/25054), 112.56 MiB | 12.61 MiB/s, done.
Resolvi

In [0]:
import pandas as pd
import requests
import numpy as np

In [3]:
# World confirmed
data = pd.read_csv("COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv")
data['Country/Region'] = data['Country/Region'].apply(lambda x: str(x))
data['Province/State'] = data['Province/State'].apply(lambda x: str(x))
data = data.groupby(["Country/Region", "Province/State"]).sum().reset_index()
del data["Lat"]
del data["Long"]
data.columns = list(data.columns[:2]) + [pd.to_datetime(i).strftime("%m/%d/%y") for i in data.columns[2:]]

# World deaths
data_deaths = pd.read_csv("COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv")
data_deaths['Country/Region'] = data_deaths['Country/Region'].apply(lambda x: str(x))
data_deaths['Province/State'] = data_deaths['Province/State'].apply(lambda x: str(x))
data_deaths = data_deaths.groupby(["Country/Region", "Province/State"]).sum().reset_index()
del data_deaths["Lat"]
del data_deaths["Long"]
data_deaths.columns = list(data_deaths.columns[:2]) + [pd.to_datetime(i).strftime("%m/%d/%y") for i in data_deaths.columns[2:]]

# US confirmed
data_US = pd.read_csv("COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv")
for col in ["UID", "iso2","iso3","code3","FIPS","Admin2", "Combined_Key"]:
  del data_US[col]
data_US = data_US.rename(columns={"Long_":"Long", "Country_Region":"Country/Region","Province_State":"Province/State"})
data_US['Country/Region'] = data_US['Country/Region'].apply(lambda x: str(x))
data_US['Province/State'] = data_US['Province/State'].apply(lambda x: str(x))
data_US = data_US.groupby(["Country/Region", "Province/State"]).sum().reset_index()
del data_US["Lat"]
del data_US["Long"]
data_US.columns = list(data_US.columns[:2]) + [pd.to_datetime(i).strftime("%m/%d/%y") for i in data_US.columns[2:]]

# US deaths
data_deaths_US = pd.read_csv("COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv")
for col in ["UID", "iso2","iso3","code3","FIPS","Admin2", "Combined_Key", "Population"]:
  del data_deaths_US[col]
data_deaths_US = data_deaths_US.rename(columns={"Long_":"Long", "Country_Region":"Country/Region","Province_State":"Province/State"})
data_deaths_US['Country/Region'] = data_deaths_US['Country/Region'].apply(lambda x: str(x))
data_deaths_US['Province/State'] = data_deaths_US['Province/State'].apply(lambda x: str(x))
data_deaths_US = data_deaths_US.groupby(["Country/Region", "Province/State"]).sum().reset_index()
del data_deaths_US["Lat"]
del data_deaths_US["Long"]
data_deaths_US.columns = list(data_deaths_US.columns[:2]) + [pd.to_datetime(i).strftime("%m/%d/%y") for i in data_deaths_US.columns[2:]]

data = data.append(data_US)
data_deaths = data_deaths.append(data_deaths_US)

# Today data countries
data_today = pd.read_csv("COVID-19-new/data/cases_country.csv")

data_today_deaths = data_today[["Country_Region", "Lat", "Long_", "Deaths"]]
data_today_deaths["Province/State"] = 'nan'
data_today_deaths = data_today_deaths.rename(columns={"Long_":"Long", "Deaths":pd.to_datetime('today').strftime("%m/%d/%y"), "Country_Region":"Country/Region"})
data_today_deaths['Country/Region'] = data_today_deaths['Country/Region'].apply(lambda x: str(x))
data_today_deaths['Province/State'] = data_today_deaths['Province/State'].apply(lambda x: str(x))
data_today_deaths = data_today_deaths.groupby(["Country/Region", "Province/State"]).sum().reset_index()
del data_today_deaths["Lat"]
del data_today_deaths["Long"]
data_today_deaths.columns = list(data_today_deaths.columns[:2]) + [pd.to_datetime(i).strftime("%m/%d/%y") for i in data_today_deaths.columns[2:]]

data_today_confirmed = data_today[["Country_Region", "Lat", "Long_", "Confirmed"]]
data_today_confirmed["Province/State"] = 'nan'
data_today_confirmed = data_today_confirmed.rename(columns={"Long_":"Long", "Confirmed":pd.to_datetime('today').strftime("%m/%d/%y"), "Country_Region":"Country/Region"})
data_today_confirmed['Country/Region'] = data_today_confirmed['Country/Region'].apply(lambda x: str(x))
data_today_confirmed['Province/State'] = data_today_confirmed['Province/State'].apply(lambda x: str(x))
data_today_confirmed = data_today_confirmed.groupby(["Country/Region", "Province/State"]).sum().reset_index()
del data_today_confirmed["Lat"]
del data_today_confirmed["Long"]
data_today_confirmed.columns = list(data_today_confirmed.columns[:2]) + [pd.to_datetime(i).strftime("%m/%d/%y") for i in data_today_confirmed.columns[2:]]

# Today data states
states_today = pd.read_csv("COVID-19-new/data/cases_state.csv")

states_today_deaths = states_today[["Country_Region", "Province_State", "Lat", "Long_", "Deaths"]]
states_today_deaths = states_today_deaths.rename(columns={"Long_":"Long", "Deaths":pd.to_datetime('today').strftime("%m/%d/%y"), "Country_Region":"Country/Region", "Province_State":"Province/State"})
states_today_deaths['Country/Region'] = states_today_deaths['Country/Region'].apply(lambda x: str(x))
states_today_deaths['Province/State'] = states_today_deaths['Province/State'].apply(lambda x: str(x))
states_today_deaths = states_today_deaths.groupby(["Country/Region", "Province/State"]).sum().reset_index()
del states_today_deaths["Lat"]
del states_today_deaths["Long"]
states_today_deaths.columns = list(states_today_deaths.columns[:2]) + [pd.to_datetime(i).strftime("%m/%d/%y") for i in states_today_deaths.columns[2:]]

states_today_confirmed = states_today[["Country_Region", "Province_State", "Lat", "Long_", "Confirmed"]]
states_today_confirmed = states_today_confirmed.rename(columns={"Long_":"Long", "Confirmed":pd.to_datetime('today').strftime("%m/%d/%y"), "Country_Region":"Country/Region", "Province_State":"Province/State"})
states_today_confirmed['Country/Region'] = states_today_confirmed['Country/Region'].apply(lambda x: str(x))
states_today_confirmed['Province/State'] = states_today_confirmed['Province/State'].apply(lambda x: str(x))
states_today_confirmed = states_today_confirmed.groupby(["Country/Region", "Province/State"]).sum().reset_index()
del states_today_confirmed["Lat"]
del states_today_confirmed["Long"]
states_today_confirmed.columns = list(states_today_confirmed.columns[:2]) + [pd.to_datetime(i).strftime("%m/%d/%y") for i in states_today_confirmed.columns[2:]]

data_today_confirmed = data_today_confirmed.append(states_today_confirmed)
data_today_deaths = data_today_deaths.append(states_today_deaths)

data = data.merge(data_today_confirmed, how='left', on=['Country/Region','Province/State'])
data_deaths = data_deaths.merge(data_today_deaths, how='left', on=['Country/Region','Province/State'])
data.to_csv('covid-19-solution/data/data_start/_data_confirmed.csv', index=False)
data_deaths.to_csv('covid-19-solution/data/data_start/_data_deaths.csv', index=False)
data.sample(3)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,Country/Region,Province/State,01/22/20,01/23/20,01/24/20,01/25/20,01/26/20,01/27/20,01/28/20,01/29/20,01/30/20,01/31/20,02/01/20,02/02/20,02/03/20,02/04/20,02/05/20,02/06/20,02/07/20,02/08/20,02/09/20,02/10/20,02/11/20,02/12/20,02/13/20,02/14/20,02/15/20,02/16/20,02/17/20,02/18/20,02/19/20,02/20/20,02/21/20,02/22/20,02/23/20,02/24/20,02/25/20,02/26/20,02/27/20,02/28/20,...,03/29/20,03/30/20,03/31/20,04/01/20,04/02/20,04/03/20,04/04/20,04/05/20,04/06/20,04/07/20,04/08/20,04/09/20,04/10/20,04/11/20,04/12/20,04/13/20,04/14/20,04/15/20,04/16/20,04/17/20,04/18/20,04/19/20,04/20/20,04/21/20,04/22/20,04/23/20,04/24/20,04/25/20,04/26/20,04/27/20,04/28/20,04/29/20,04/30/20,05/01/20,05/02/20,05/03/20,05/04/20,05/05/20,05/06/20,05/07/20
151,Jamaica,nan,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,32,36,36,44,47,47,53,58,58,63,63,63,63,65,69,73,73,125,143,143,163,173,223,223,233,257,288,305,350,364,364,396,422,432,463,469,471,473,478,478
133,Greece,nan,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,3,4,...,1156,1212,1314,1415,1544,1613,1673,1735,1755,1832,1884,1955,2011,2081,2114,2145,2170,2192,2207,2224,2235,2235,2245,2401,2408,2463,2490,2506,2517,2534,2566,2576,2591,2612,2620,2626,2632,2642,2663,2663
300,US,New Jersey,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,13386,16636,18696,22255,25590,29895,34124,37505,41090,44416,47437,51027,54588,58151,61850,64584,68824,71030,75317,78467,81420,85301,88722,92387,95914,100025,102196,105498,109038,111188,113856,116365,118652,121190,123717,126744,128269,130593,131890,131890


In [4]:
# Russia confirmed
data_rus = pd.read_csv("COVID-19_plus_Russia/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_RU.csv")
for col in ["UID", "iso2","iso3","code3","FIPS","Admin2", "Combined_Key"]:
  del data_rus[col]
data_rus = data_rus.rename(columns={"Long_":"Long", "Country_Region":"Country/Region","Province_State":"Province/State"})
data_rus['Country/Region'] = data_rus['Country/Region'].apply(lambda x: str(x))
data_rus['Province/State'] = data_rus['Province/State'].apply(lambda x: str(x))
data_rus = data_rus.groupby(["Country/Region", "Province/State"]).sum().reset_index()
data_rus['Province/State'] = data_rus['Province/State'].apply(lambda x: x if (x != 'Altay republic') else 'Republic of Altay')
del data_rus["Lat"]
del data_rus["Long"]
data_rus.columns = list(data_rus.columns[:2]) + [pd.to_datetime(i).strftime("%m/%d/%y") for i in data_rus.columns[2:]]

# Russia deaths
data_deaths_rus = pd.read_csv("COVID-19_plus_Russia/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_RU.csv")
for col in ["UID", "iso2","iso3","code3","FIPS","Admin2", "Combined_Key"]:
  del data_deaths_rus[col]
data_deaths_rus = data_deaths_rus.rename(columns={"Long_":"Long", "Country_Region":"Country/Region","Province_State":"Province/State"})
data_deaths_rus['Country/Region'] = data_deaths_rus['Country/Region'].apply(lambda x: str(x))
data_deaths_rus['Province/State'] = data_deaths_rus['Province/State'].apply(lambda x: str(x))
data_deaths_rus = data_deaths_rus.groupby(["Country/Region", "Province/State"]).sum().reset_index()
data_deaths_rus['Province/State'] = data_deaths_rus['Province/State'].apply(lambda x: x if (x != 'Altay republic') else 'Republic of Altay')
del data_deaths_rus["Lat"]
del data_deaths_rus["Long"]
data_deaths_rus.columns = list(data_deaths_rus.columns[:2]) + [pd.to_datetime(i).strftime("%m/%d/%y") for i in data_deaths_rus.columns[2:]]

# Data today (from coronavirus-monitor.ru)
class window():
  pass
true = True
false = False
null = np.nan
response = requests.get('https://coronavirus-monitor.ru/jquery-lite-9.js')
result = response.text.strip()
exec(result)
date = window.dataFromServer['russianSubjects']['data']['subjects'][0]['statistics'][-1]['date']

countries_dict = {
' Kabardino-Balkarian Republic': 'Republic of Kabardino-Balkaria',' Kurgan region': 'Kurgan oblast',
'Adygeya Republic': 'Republic of Adygeia','Altai Krai': 'Altayskiy kray',
'Amur oblast': 'Amursk oblast','Arkhangelsk region': 'Arkhangelsk oblast',
'Astrakhan oblast': 'Astrahan oblast','Bashkortostan': 'Republic of Bashkortostan',
'Chechen Republic ': 'Chechen republic','Chelyabinsk region': 'Cheliabinsk oblast',
'Chukotka Autonomous Okrug': 'Chukotskiy autonomous oblast','Chuvash Republic': 'Republic of Chuvashia',
'Irkutsk oblast': 'Irkutsk oblast','Ivanovo region': 'Ivanovo oblast',
'Jewish autonomous oblast': 'Jewish Autonomous oblast','Kaluga region': 'Kaluga oblast',
'Kamchatka Krai': 'Kamchatskiy kray','Karachay-Cherkess Republic': 'Republic of Karachaevo-Cherkessia',
'Khabarovsk territory': 'Habarovskiy kray','Khanty-Mansi Autonomous Okrug': 'Hanty-Mansiyskiy AO',
'Kirov region': 'Kirov oblast','Kostroma oblast': 'Kostroma oblast',
'Krasnodar region': 'Krasnodarskiy kray','Krasnoyarsk region': 'Krasnoyarskiy kray',
'Kursk oblast': 'Kursk oblast','Leningrad region': 'Leningradskaya oblast',
'Magadan oblast': 'Magadan oblast','Moscow': 'Moscow',
'Moscow oblast': 'Moscow oblast','Murmansk region': 'Murmansk oblast',
'Nenets Autonomous Okrug': 'Nenetskiy autonomous oblast','Nizhny Novgorod region': 'Nizhegorodskaya oblast',
'Novgorod region': 'Novgorod oblast','Novosibirsk region': 'Novosibirsk oblast',
'Omsk oblast': 'Omsk oblast','Orel region': 'Orel oblast',
'Orenburg region': 'Orenburg oblast','Penza region': 'Pensa oblast',
'Perm region': 'Perm oblast','Pskov Oblast': 'Pskov oblast',
'Republic Of Khakassia': 'Republic of Hakassia','Republic of Buryatia': 'Republic of Buriatia',
'Republic of Crimea': 'Republic of Crimea','Republic of Dagestan': 'Republic of Dagestan',
'Republic of Kalmykia': 'Republic of Kalmykia','Republic of Karelia': 'Republic of Karelia',
'Republic of Mari El': 'Republic of Mariy El','Republic of Mordovia': 'Republic of Mordovia',
'Rostov oblast': 'Rostov oblast','Ryazan region': 'Ryazan oblast',
'Saint Petersburg': 'Saint Petersburg','Sakhalin oblast': 'Sakhalin oblast',
'Samara region': 'Samara oblast','Saratov region': 'Saratov oblast',
'Sevastopol': 'Sevastopol','Smolensk oblast': 'Smolensk oblast',
'Sverdlovsk region': 'Sverdlov oblast','Tambov region': 'Tambov oblast',
'The Republic of Ingushetia': 'Ingushetia republic','The Republic of North Ossetia – Alania ': 'Republic of North Osetia - Alania',
'Tomsk region': 'Tomsk oblast','Transbaikal territory': 'Zabaykalskiy kray',#
'Tula region': 'Tula oblast','Tuva Republic': 'Republic of Tyva',
'Tver region': 'Tver oblast','Tyumen region': 'Tumen oblast',
'Udmurt Republic': 'Republic of Udmurtia','Ulyanovsk region': 'Ulianovsk oblast',
'Vladimir region': 'Vladimir oblast','Volgograd oblast': 'Volgograd oblast',
'Vologda oblast': 'Vologda oblast','Voronezh region': 'Voronezh oblast',
'Yakutia': 'Saha republic','Yamalo-Nenets Autonomous district': 'Yamalo-Nenetskiy AO',
'Yaroslavl oblast': 'Yaroslavl oblast','Белгородская область': 'Belgorod oblast',
'Брянская область': 'Briansk oblast','Калининградская область': 'Kaliningrad oblast',
'Кемеровская область':  'Kemerovo oblast','Липецкая область': 'Lipetsk oblast',
'Приморский край': 'Primorskiy kray','Республика Коми': 'Komi republic',
'Ставропольский край': 'Stavropolskiy kray','Татарстан': 'Republic of Tatarstan',
'Республика Алтай': 'Republic of Altay',
}

dict_russia_conf = {}
dict_russia_dea = {}
for v in window.dataFromServer['russianSubjects']['data']['subjects']:
  dict_russia_conf[v['en']] = v['statistics'][-1]['confirmed']
  dict_russia_dea[v['en']] = v['statistics'][-1]['deaths']

# Confirmed
dict_russia_conf_main = []
for n in dict_russia_conf:
  dict_russia_conf_main.append([countries_dict[n], dict_russia_conf[n]])
dict_russia_conf_main = pd.DataFrame(np.array(dict_russia_conf_main))
dict_russia_conf_main.columns = ['Province/State',pd.to_datetime('today').strftime("%m/%d/%y")]
dict_russia_conf_main['Country/Region'] = 'Russia'
dict_russia_conf_main['Country/Region'] = dict_russia_conf_main['Country/Region'].apply(lambda x: str(x))
dict_russia_conf_main['Province/State'] = dict_russia_conf_main['Province/State'].apply(lambda x: str(x))
dict_russia_conf_main = dict_russia_conf_main.groupby(["Country/Region", "Province/State"]).sum().reset_index()
dict_russia_conf_main.columns = list(dict_russia_conf_main.columns[:2]) + [pd.to_datetime(i).strftime("%m/%d/%y") for i in dict_russia_conf_main.columns[2:]]

# Deaths
dict_russia_dea_main = []
for n in dict_russia_dea:
  dict_russia_dea_main.append([countries_dict[n], dict_russia_dea[n]])
dict_russia_dea_main = pd.DataFrame(np.array(dict_russia_dea_main))
dict_russia_dea_main.columns = ['Province/State',pd.to_datetime('today').strftime("%m/%d/%y")]
dict_russia_dea_main['Country/Region'] = 'Russia'
dict_russia_dea_main['Country/Region'] = dict_russia_dea_main['Country/Region'].apply(lambda x: str(x))
dict_russia_dea_main['Province/State'] = dict_russia_dea_main['Province/State'].apply(lambda x: str(x))
dict_russia_dea_main = dict_russia_dea_main.groupby(["Country/Region", "Province/State"]).sum().reset_index()
dict_russia_dea_main.columns = list(dict_russia_dea_main.columns[:2]) + [pd.to_datetime(i).strftime("%m/%d/%y") for i in dict_russia_dea_main.columns[2:]]

dict_russia_conf_main[dict_russia_conf_main.columns[-1]] = dict_russia_conf_main[dict_russia_conf_main.columns[-1]].astype('int64')
dict_russia_dea_main[dict_russia_dea_main.columns[-1]] = dict_russia_dea_main[dict_russia_dea_main.columns[-1]].astype('int64')

data_rus = data_rus.merge(dict_russia_conf_main, how='left', on=['Country/Region','Province/State'])
data_deaths_rus = data_deaths_rus.merge(dict_russia_dea_main, how='left', on=['Country/Region','Province/State'])
data_rus.to_csv('covid-19-solution/data/data_start/_data_confirmed_rus.csv', index=False)
data_deaths_rus.to_csv('covid-19-solution/data/data_start/_data_deaths_rus.csv', index=False)
data_rus.sample(3)

,Country/Region,Province/State,01/22/20,01/23/20,01/24/20,01/25/20,01/26/20,01/27/20,01/28/20,01/29/20,01/30/20,01/31/20,02/01/20,02/02/20,02/03/20,02/04/20,02/05/20,02/06/20,02/07/20,02/08/20,02/09/20,02/10/20,02/11/20,02/12/20,02/13/20,02/14/20,02/15/20,02/16/20,02/17/20,02/18/20,02/19/20,02/20/20,02/21/20,02/22/20,02/23/20,02/24/20,02/25/20,02/26/20,02/27/20,02/28/20,...,03/29/20,03/30/20,03/31/20,04/01/20,04/02/20,04/03/20,04/04/20,04/05/20,04/06/20,04/07/20,04/08/20,04/09/20,04/10/20,04/11/20,04/12/20,04/13/20,04/14/20,04/15/20,04/16/20,04/17/20,04/18/20,04/19/20,04/20/20,04/21/20,04/22/20,04/23/20,04/24/20,04/25/20,04/26/20,04/27/20,04/28/20,04/29/20,04/30/20,05/01/20,05/02/20,05/03/20,05/04/20,05/05/20,05/06/20,05/07/20
78,Russia,Vladimir oblast,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,2,2,3,3,3,3,3,5,12,14,38,42,48,71,84,100,145,159,183,198,225,259,280,293,339,381,426,475,518,597,669,669,715,775,830,896,944,1001,1079
75,Russia,Tumen oblast,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,4,4,6,6,6,11,11,11,11,18,18,19,22,30,39,53,67,85,112,146,185,227,254,290,319,341,357,374,386,399,404,435,435,442,458,495,561,589,658,683
51,Russia,Republic of Kabardino-Balkaria,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,4,6,6,8,8,8,8,8,8,13,23,23,27,30,35,50,50,50,69,92,114,136,152,170,193,218,243,272,300,329,367,456,456,499,544,634,739,820,925,1008


In [5]:
pd.read_csv('timeseries.csv').to_csv('covid-19-solution/data/other/timeseries.csv')
pd.read_csv('https://coronadatascraper.com/timeseries-jhu.csv').to_csv('covid-19-solution/data/other/timeseries-jhu.csv')

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [0]:
data_all = pd.read_csv('https://coronadatascraper.com/timeseries-jhu.csv')
data_all.drop(['level', 'city', 'county', 'state', 'lat', 'long', 'population', 'url', 'aggregate', 'tz'], axis=1, inplace=True)
data_all = data_all.rename(columns={'country':'Country/Region', 'name':'Province/State'})
data_all.columns = list(data_all.columns[:2]) + [pd.to_datetime(i).strftime("%m/%d/%y") for i in data_all.columns[2:]]

In [10]:
data_all.shape

(4555, 108)

In [0]:
data_all2 = []
data_all_cols = data_all.columns
for v in data_all.values:
  if len(np.unique(v[2:])) > 40:
    data_all2.append(v)
data_all2 = pd.DataFrame(data_all2)
data_all2.columns = data_all_cols

In [18]:
data_all2.shape

(1773, 108)

In [0]:
data2 = data.append(data_all2)
data2 = data2.fillna(0)
data2.to_csv('covid-19-solution/data/data_start/_data_confirmed_all.csv')